In [1]:
from daily_data import daily_data
print(len(daily_data))

720


In [2]:
from collections import namedtuple
Day = namedtuple('Day', ['total_payout', 'total_shares', 'unclaimed_sats', 'payout_acc'])

In [3]:
def decode_raw_day(day):
    bytes_day = day.to_bytes(32, 'big')
    total_payout = int.from_bytes(bytes_day[-9:], 'big')
    total_shares = int.from_bytes(bytes_day[-18:-9], 'big')
    unclaimed_sats = int.from_bytes(bytes_day[:-18], 'big')
    return total_payout, total_shares, unclaimed_sats


In [4]:
def require_is_uint256(x):
    if type(x) is not int:
        raise TypeError('x not of type int')
    if x >= (1 << 256):
        raise ValueError('x exceeds uint256 size')
    return x


In [5]:
def decode_and_calc_day_data(acc_scale):
    day_data = []
    next_acc = 0
    for raw_day in daily_data[1:]:
        total_payout, total_shares, unclaimed_sats = decode_raw_day(raw_day)
        day_data.append(Day(total_payout, total_shares, unclaimed_sats, next_acc))
        if total_shares > 0:
            next_acc += require_is_uint256(total_payout * acc_scale // total_shares)
    day_data.append(Day(None, None, None, next_acc))
    return day_data, len(day_data) - 1

In [6]:
from random import random, randint, seed
import math

In [7]:
math.log(100_000 / 1, 10)

5.0

In [8]:
def randrange(low, high=None):
    if high is None:
        high = low
        low = 0
    return (high - low) * random() + low

In [9]:
def lin_random_bn(low, high, s=1e12):
    return randint(int(low * s), int(high * s))

def log_random_bn(low, high, s=1e12):
    log_diff = math.log(high / low)
    return int(low * math.exp(randrange(log_diff)) * s)

def rand_sign(neg_prob):
    if random() <= neg_prob:
        return -1
    return 1
        

In [10]:
def full_calculate_payout(day_data, user_shares, start_day, end_day, extra_scale=1):
    return sum(
        user_shares * total_payout * extra_scale // total_shares
        for total_payout, total_shares, _, _ in day_data[start_day:end_day]
    ) // extra_scale

In [15]:
def acc_calculate_payout(day_data, acc_scale, user_shares, start_day, end_day):
    start_acc = day_data[start_day].payout_acc
    end_acc = day_data[end_day].payout_acc
    return user_shares * (end_acc - start_acc) // acc_scale
    

In [34]:
def disp_hearts(hearts):
    sign = '' if hearts <= 0 else '+'
    if abs(hearts) <= 10**6: # 0.01 HEX
        return f'{sign}{hearts:,} Hearts'
    return f'{sign}{hearts / 1e8:,.2f} Hex'

def disp_perc(perc):
    if abs(perc) < 1e-4:
        return perc
    return f'{perc:.2%}'
    

In [35]:
def calculate_errors(day_data_scale, user_shares, start_day, end_day, extra_precision):
    day_data, _ = decode_and_calc_day_data(day_data_scale)
    span = end_day - start_day
    full_payout = full_calculate_payout(day_data, user_shares, start_day, end_day, extra_precision)

    acc_payout = acc_calculate_payout(day_data, day_data_scale, user_shares, start_day, end_day)

    acc_error = acc_payout / full_payout - 1
    abs_acc_error = acc_payout - full_payout
    print(f'\nerror at x{float(extra_precision)} added precision:')
    print(f'calculation error: {disp_perc(acc_error)} ({disp_hearts(abs_acc_error)})')

    relative_error = abs_acc_error / span
    print(f'relative error: {disp_hearts(relative_error)} / Day')
    

In [42]:
# RUN this cell to simulate single stake

base_scale = 10 ** 18
full_daily_data, total_days = decode_and_calc_day_data(base_scale)

user_shares = log_random_bn(1e2, 1e6)
print(f'user shares: {user_shares / 1e12:,.6f} T-Shares')

maximize_range = False

if maximize_range:
    start_day = randint(0, 10)
    end_day = randint(total_days - 10, total_days)
else:
    start_day = randint(0, total_days - 1)
    end_day = randint(start_day + 1, total_days)
span = end_day - start_day
print(f'day {start_day} - {end_day} (span: {span})')

full_payout = full_calculate_payout(full_daily_data, user_shares, start_day, end_day)
print(f'full payout: {disp_hearts(full_payout)}')

acc_payout = acc_calculate_payout(full_daily_data, base_scale, user_shares, start_day, end_day)
print(f'accumulated payout: {disp_hearts(acc_payout)}')

for scale_exp in range(12, 25):
    scale = 10**scale_exp
    print(f'\n\nMain accumulator scale: {scale:,} ({scale_exp})')
    params = user_shares, start_day, end_day
    calculate_errors(scale, *params, 10**0)
    calculate_errors(scale, *params, 10**1)
    calculate_errors(scale, *params, 10**2)
    calculate_errors(scale, *params, 10**3)
    calculate_errors(scale, *params, 10**4)
    calculate_errors(scale, *params, 10**6)


user shares: 8,159.463640 T-Shares
day 191 - 695 (span: 504)
full payout: +21,519,205.07 Hex
accumulated payout: +21,519,205.07 Hex


Main accumulator scale: 1,000,000,000,000 (12)

error at x1.0 added precision:
calculation error: -8.868571432785188e-10 (-0.02 Hex)
relative error: -3,786.5992063492063 Hearts / Day

error at x10.0 added precision:
calculation error: -8.869626144658582e-10 (-0.02 Hex)
relative error: -3,787.0496031746034 Hearts / Day

error at x100.0 added precision:
calculation error: -8.869732726068946e-10 (-0.02 Hex)
relative error: -3,787.095238095238 Hearts / Day

error at x1000.0 added precision:
calculation error: -8.869741607853143e-10 (-0.02 Hex)
relative error: -3,787.0992063492063 Hearts / Day

error at x10000.0 added precision:
calculation error: -8.869741607853143e-10 (-0.02 Hex)
relative error: -3,787.0992063492063 Hearts / Day

error at x1000000.0 added precision:
calculation error: -8.869741607853143e-10 (-0.02 Hex)
relative error: -3,787.0992063492063 H